*** Module 3: Waves ***

# Sods Shock tube, Richtmyer method #

In [91]:
import numpy
from matplotlib import pyplot
%matplotlib inline

from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['font.size']=16

import sympy
from sympy import init_printing
init_printing()
from sympy.utilities.lambdify import lambdify

In [109]:
def vector_u(rho,u,p):
    # there is a little problem because the name of the vector is the same as the name of the velocity variable 
    #we change it into vector_u but it is a bit cumbersome
    v=numpy.zeros((3,len(rho)))
    v=[rho,rho*u,rho*(p/(0.4*rho)+0.5*(u)**2)]
    return v

In [112]:
def computef(uv):
    # We define f as a function of u
    f=numpy.zeros_like((uv))
    f[0]=uv[1]
    f[1]=uv[1]**2/uv[0]+0.4*(uv[2]-0.5*uv[1]**2/uv[0])
    f[2]=(uv[1]/uv[0])*(uv[2]+0.4*(uv[2]-0.5*uv[1]**2/uv[0]))
    return f

In [113]:
dx= 0.25 #meter
dt=0.0002 #second
nx=81 #so that we cover 20 km, from -10 to +10

In [114]:
x=numpy.linspace(-10,10,nx)

In [184]:
IC=numpy.zeros((3,nx))
IC[0,:40]=1.0 #kg/m³
IC[0,40:]=0.125 #kg/m³
IC[1,0:]=0.0 #m/s
IC[2,:40]=100.0 #kN/m²
IC[2,40:]=10.0 #kN/m²
print(IC)

[[   1.       1.       1.       1.       1.       1.       1.       1.
     1.       1.       1.       1.       1.       1.       1.       1.
     1.       1.       1.       1.       1.       1.       1.       1.
     1.       1.       1.       1.       1.       1.       1.       1.
     1.       1.       1.       1.       1.       1.       1.       1.
     0.125    0.125    0.125    0.125    0.125    0.125    0.125    0.125
     0.125    0.125    0.125    0.125    0.125    0.125    0.125    0.125
     0.125    0.125    0.125    0.125    0.125    0.125    0.125    0.125
     0.125    0.125    0.125    0.125    0.125    0.125    0.125    0.125
     0.125    0.125    0.125    0.125    0.125    0.125    0.125    0.125
     0.125]
 [   0.       0.       0.       0.       0.       0.       0.       0.
     0.       0.       0.       0.       0.       0.       0.       0.
     0.       0.       0.       0.       0.       0.       0.       0.
     0.       0.       0.       0.       0.       

In [185]:
def predictor(v,f):
    #fonction qui prend u au temps n et f au temps n et qui donne une liste u en n+1/2 avec les indices 
    #d'espace décalés de 1/2 vers la droite (i+1/2=i)
    uu=numpy.zeros((3,81))
    ff=numpy.zeros((3,81))
    uu[:,:]=v.copy()
    ff[:,:]=f.copy()
    for i in range (0,2):
        uui=uu[i]
        vi=v[i]
        fi=f[i]
        uui[1:]=0.5*(vi[1:]+vi[:-1])-0.5*dt/dx*(fi[1:]-fi[:-1])
        ff=computef(uu)
    return ff

In [186]:
def Shock(v):
    uu=numpy.zeros((3,81))
    uu[:,:]=v.copy()
    t=0
    while t<50:
        ff=computef(uu)
        for i in range (0,2):
            vi=v[i]
            uui=uu[i]
            predictori=predictor(uu,ff)[i]
            vi[1:]=uui[:-1]-dt/dx*(predictori[1:]-predictori[:-1])
            v[i]=vi
        uu=v.copy()
        t=t+1
    return v

In [187]:
v=vector_u(IC[0],IC[1],IC[2])
f=computef(v)
predictor(v,f)
Shock(v)

[array([ 1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         0.9999424,  0.9999424,  0.9999424,  0.9999424,  0.9999424,
         0.9999424,  0.9999424,  0.9999424,  0.9999424,  0.9999424,
         0.9999424,  0.9999424,  0.9999424,  0.9999424,  0.9999424,
         0.9999424,  0.9999424,  0.9999424,  0.9999424,  0.9999424,
         0.9999424,  0.9999424,  0.9999424,  0.9999424,  0.9999424,
         0.9999424,  0.9999424,  0.9999424,  0.9999424,  0.9999424,
         0.9999424,  0.9999424,  0.9999424,  0.9